In [65]:
# Data Collection

!pip install kaggle --upgrade
!pip3 install Pillow

import kaggle
import os
import json
import random
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS

def get_colors(image_file, numcolors=3, resize=150, output = []):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))
 
    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)
 
    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))
    return colors

In [82]:
## You have to collect and download a set of images. You have the following tasks to program, automating the process as much as possible:

### 1. Create a folder called images.

!mkdir -p ./images
!mkdir -p ./metadata

### 2. Download open-licensed images to the folder images (minimum 100 images).

kaggle.api.authenticate()
kaggle.api.dataset_download_files('nielspace/pexels-mountain-images', path='./images', unzip=True)    
    
### 3. Save metadata of every image like image size, image format (.jpeg, .png, etc.), image orientation (landscape, portrait, square, etc.), creation date, camera model, etc. in one or more JSON files. You can make use of the Exif information present in the image files.

directory = './images/Mountain'
metadata = {}
orientation = ""

for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        path, file_extension = os.path.splitext(f)
        time_creation = os.path.getmtime(f)
        size = Image.open(f).size
        if size[0]>size[1] :
            orientation = "landscape"
        elif size[0] == size[1] :
            orientation = "square"
        else :
            orientation = "portrait"
        metadata[filename] = {
            "file_extension": file_extension,
            "creation_date": datetime.fromtimestamp(time_creation/1000.0).strftime("%m/%d/%Y, %H:%M:%S"),
            "size": size,
            "orientation": orientation,
            "colors": get_colors(directory + "/" + filename)
            }

with open("./metadata/metadata.json", "w") as outfile:
    json.dump(metadata, outfile, indent=4)

        

In [84]:
# Labeling and Annotation

directory = './images/Mountain'
data_path = "./metadata/metadata.json"



if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)
        
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        tags = {}
        tags["like"] = random.randint(0,100)
        tags["hashtag"] = "moutain"
        rand = random.randint(1, 3)
        if rand == 1 :
            tags["test"] = "yes"
        else :
            tags["test"] = "no"
        json_data[filename]["tags"] = tags
        
with open("./metadata/metadata.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [87]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Data Analyses

numb_users = 1
directory = './images/Mountain'
data_path = "./metadata/metadata.json"
rand = 0

# Metadata for each file
if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)

# Creating data for each user
json_data_users = {}        
for i in range (0, numb_users):
    images_per_users = []
    tags_per_users = []
    rand = random.randint(1, 4)
    if rand == 1:
        tags_per_users.append(["like", "colors"])
    elif rand == 2:
        tags_per_users.append("colors")
    elif rand == 3:
        tags_per_users.append(["hashtag", "colors"])
    else:
        tags_per_users.append(["like, hashtag", "colors"])
    for filename in os.listdir(directory) :
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            rand = random.randint(1, 3)
            if rand == 1 :
                if json_data[filename]["tags"]["test"] == "yes":
                    images_per_users.append(filename)
    json_data_users[i] = {
        "images": images_per_users,
        "tags": tags_per_users
    }
    

# Creating dataframes to predict what the user might like
data = []
result = []
json_data_learning = dict(list(json_data.items())[len(json_data)//3:])
for i in json_data_learning:
    if i in json_data_users[0]["images"]:
        result.append('Favorite')
    else:
        result.append('NotFavorite')
    data.append([json_data_learning[i]["colors"][0][0],
                 json_data_learning[i]["colors"][0][1],
                 json_data_learning[i]["colors"][0][2],
                 json_data_learning[i]["orientation"], 
                 json_data_learning[i]["size"][0],
                 json_data_learning[i]["size"][1],
                 json_data_learning[i]["tags"]["test"]
                ])

dataframe = pd.DataFrame(data, columns=['red', 'green', 'blue', 'orientation', 'width', 'height', 'test'])
resultframe = pd.DataFrame(result, columns=['favorite'])

print(dataframe)
print(resultframe)

#generating numerical labels
le1 = LabelEncoder()
dataframe['orientation'] = le1.fit_transform(dataframe['orientation'])

le2 = LabelEncoder()
dataframe['test'] = le2.fit_transform(dataframe['test'])

le3 = LabelEncoder()
resultframe['favorite'] = le3.fit_transform(resultframe['favorite'])

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

prediction = dtc.predict([
        [1,
         2,
         3,
         le1.transform(['landscape'])[0],
         12,
         23,
         le2.transform(['yes'])[0]
        ]])
print(le3.inverse_transform(prediction))
print(dtc.feature_importances_)

    red  green  blue orientation  width  height test
0   250    239   238    portrait   3648    5472   no
1   249    216   215    portrait   3648    5472   no
2    82     69    57    portrait   3605    5438   no
3    24     64    62   landscape   5266    3403   no
4    59     62    72   landscape   6914    4463   no
..  ...    ...   ...         ...    ...     ...  ...
63  157    191   210   landscape   3969    2729   no
64   61     54    60    portrait   2322    2902  yes
65   38     51    56   landscape   5746    3232   no
66  208    221   231   landscape   4110    2597   no
67  181    187   205    portrait   5440    8160  yes

[68 rows x 7 columns]
       favorite
0   NotFavorite
1   NotFavorite
2   NotFavorite
3   NotFavorite
4   NotFavorite
..          ...
63  NotFavorite
64     Favorite
65  NotFavorite
66  NotFavorite
67  NotFavorite

[68 rows x 1 columns]
['Favorite']
[0.         0.13148629 0.26984127 0.         0.37821388 0.
 0.22045855]
